#### First you need to create an account at [ngrok](https://ngrok.com/). Then you will need your ***Your Authtoken***.

In [16]:
!pip install streamlit==1.15.2 mlflow pyngrok tf-keras-vis -q

In [17]:
!pip install git+https://github.com/vafaei-ar/ahunt-app -q --ignore-installed
!pkill ahunt

  Preparing metadata (setup.py) ... done


In [18]:
import PIL
import uuid
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from shutil import copy
from pyngrok import ngrok
from getpass import getpass

# Galaxy10 DECals Dataset

Galaxy10 dataset (17736 images)
- Class 0 (1081 images): Disturbed
- Class 1 (1853 images): Merging
- Class 2 (2645 images): Round Smooth
- Class 3 (2027 images): In-between Round Smooth
- Class 4 ( 334 images): Cigar Shaped Smooth
- Class 5 (2043 images): Barred Spiral
- Class 6 (1829 images): Unbarred Tight Spiral
- Class 7 (2628 images): Unbarred Loose Spiral
- Class 8 (1423 images): Edge-on without Bulge
- Class 9 (1873 images): Edge-on with Bulge

In [19]:
# !wget https://www.astro.utoronto.ca/~hleung/shared/Galaxy10/Galaxy10_DECals.h5
# import h5py
# f = h5py.File('Galaxy10_DECals.h5','r')
# images = f['images'][()]
# labels = f['ans'][()]

# filt = np.isin(labels,[1,2,4])
# x = images[filt]
# y = labels[filt]
# ndata = len(x)

# !rm -r images
# !mkdir images
# inds = np.arange(ndata)
# np.random.seed(42)
# np.random.shuffle(inds)
# flie_names = []
# for i,j in tqdm(enumerate(inds)):
#   im = PIL.Image.fromarray(x[i])
#   im.save(f'images/{j}.jpg')
    # flie_names.append(f'{j}.jpg')

# df_gt = pd.DataFrame(columns=['image','label'],index=np.arange(len(y)))
# df_gt.loc[:,'image'] = flie_names
# df_gt.loc[:,'label'] = y
# df_gt.to_csv('GroundTruth.csv',index=0)

# Rock-Paper-Scissors Images

In [20]:
# !wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
# !unzip -qq rps.zip
# !rm -r images
# !mkdir images
# n1 = 100
# n2 = n1
# n3 = 10
# flist = glob('rps/scissors/*')[:n1]+glob('rps/paper/*')[:n2]+glob('rps/rock/*')[:n3]
# y = n1*[0]+n2*[1]+n3*[2]
# flie_names = []
# for i in tqdm(flist):
#     id = uuid.uuid4().hex[:8]
#     flie_names.append(f'{id}.jpg')
#     copy(i,f'images/{id}.jpg')

# df_gt = pd.DataFrame(columns=['image','label'],index=np.arange(len(y)))
# df_gt.loc[:,'image'] = flie_names
# df_gt.loc[:,'label'] = n1*['scissor']+n2*['paper']+n3*['rock']
# df_gt.to_csv('GroundTruth.csv',index=0)

# MNIST

In [22]:
data_config = {
    0:200,
    1:200,
    2:10
}

from tensorflow import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


np.random.seed(42)

x = []
y = []
for k,v in data_config.items():
  inds = np.argwhere(y_train==k).reshape(-1)
  np.random.shuffle(inds)
  inds = inds[:v]
  x.append(x_train[inds])
  y.append(y_train[inds])

x = np.concatenate(x,axis=0)
y = np.concatenate(y,axis=0)
n_data = len(x)

!rm -r images
!mkdir images
flie_names = []
for i in tqdm(range(n_data)):
  im = PIL.Image.fromarray(x[i])
  id = uuid.uuid4().hex[:8]
  im.save(f'images/{id}_{y[i]}.jpg')
  flie_names.append(f'{id}_{y[i]}.jpg')

df_gt = pd.DataFrame(columns=['image','label'],index=np.arange(len(y)))
df_gt.loc[:,'image'] = flie_names
df_gt.loc[:,'label'] = y
df_gt.to_csv('GroundTruth.csv',index=0)

100%|██████████| 410/410 [00:00<00:00, 6246.74it/s]


In [23]:
get_ipython().system_raw('ahunt --port 8501 &')
# Terminate open tunnels if exist
ngrok.kill()
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print("running UI  at:", ngrok_tunnel.public_url)

Enter the ngrok authtoken: ··········
running UI  at: https://e080-35-221-6-74.ngrok.io


### everything is saved on "als_files" directory in the chosen image directory. 

In [ ]:
ls images/als_files

checkpoints/  idx_to_class.npy  labels.csv


In [ ]:
ls images/als_files/checkpoints/*

'images/als_files/checkpoints/labels_VGG19_13-Dec-2022_(19:16:01.040241).csv'

images/als_files/checkpoints/VGG19_encoder.tf:
assets/  keras_metadata.pb  saved_model.pb  variables/

images/als_files/checkpoints/VGG19.tf:
assets/  keras_metadata.pb  saved_model.pb  variables/


### you can also track what you did with mlflow:

In [ ]:
ls

GroundTruth.csv  images/  LOGS.log  mlruns/  rps/  rps.zip  sample_data/


In [ ]:
!pkill mlflow
# path = 'file:///content/'
# get_ipython().system_raw("mlflow ui --port 5000 --backend-store-uri '{}mlruns/' &".format(path))

get_ipython().system_raw("mlflow ui --port 5000")

ngrok.kill()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

/content/mlruns
MLflow Tracking UI: https://0823-34-147-79-76.ngrok.io
